# TP — Time Series Foundation Models (Chronos, Moirai, Time-MoE)

Ce notebook est conçu pour être exécuté **en local** ou sur **Google Colab**.

Objectif : faire un premier TP *hands-on* sur 3 modèles fondationnels pour la prévision de séries temporelles :

- **Chronos-2** (Amazon)
- **Moirai** via **Uni2TS** (Salesforce)
- **Time-MoE** (Mixture-of-Experts)

Chaque partie suit la même logique :
1) Installer / importer
2) Charger une petite série temporelle (format clair)
3) Faire une prévision + visualiser

> Conseil : sur Colab, activez un GPU (Runtime → Change runtime type → GPU).


In [ ]:
# (Optionnel) Infos environnement
import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())

try:
    import torch
    print("Torch:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("GPU:", torch.cuda.get_device_name(0))
except Exception as e:
    print("Torch not installed yet:", e)


## Données de démonstration (communes)

On utilise une **petite série univariée** (un seul signal) pour aller vite.

- En pratique : remplacez `target` par votre colonne cible.
- On crée un index temporel (`timestamp`) pour un affichage propre.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Série synthétique simple (tendance + saisonnalité + bruit)
T = 400
rng = np.random.default_rng(7)
idx = pd.date_range("2022-01-01", periods=T, freq="H")

trend = np.linspace(0, 4, T)
season = 1.2 * np.sin(2 * np.pi * np.arange(T) / 24)
noise = 0.35 * rng.standard_normal(T)

y = 10 + trend + season + noise

df = pd.DataFrame({"timestamp": idx, "target": y})
df.head()


In [ ]:
# Visualisation rapide
plt.figure(figsize=(12, 3))
plt.plot(df["timestamp"], df["target"])
plt.title("Série temporelle (démo)")
plt.xlabel("timestamp")
plt.ylabel("target")
plt.show()


---
# Partie 1 — Chronos-2 (Amazon)

Chronos fournit une interface simple via `chronos-forecasting`.

Nous allons :
- mettre les données au format `id, timestamp, target`
- charger `amazon/chronos-2`
- produire un forecast probabiliste (quantiles)


In [ ]:
# Installation (Colab / local)
!pip -q install -U chronos-forecasting matplotlib pandas


In [ ]:
import torch
from chronos import Chronos2Pipeline

# Paramètres
PRED_LEN = 48  # horizon de prévision (ex: 24, 48, 72)

# Format attendu par Chronos: id / timestamp / target
context_df = df.copy()
context_df["id"] = "ts_1"

pipeline = Chronos2Pipeline.from_pretrained(
    "amazon/chronos-2",
    device_map="cuda" if torch.cuda.is_available() else "cpu",
)

# Retourne un df contenant: timestamp, predictions, et des quantiles (ex: 0.1, 0.9)
pred_df = pipeline.predict_df(
    context_df,
    prediction_length=PRED_LEN,
    quantiles=[0.1, 0.5, 0.9],
)

pred_df.head()


In [ ]:
# Visualisation (historique + forecast)
hist = context_df.set_index("timestamp")["target"].tail(200)
pred = pred_df.set_index("timestamp")

plt.figure(figsize=(12, 3))
plt.plot(hist.index, hist.values, label="historique")
plt.plot(pred.index, pred["predictions"].values, label="prévision (médiane)")
plt.fill_between(pred.index, pred["0.1"].values, pred["0.9"].values, alpha=0.2, label="[0.1, 0.9]")

plt.title("Chronos-2 — prévision probabiliste")
plt.xlabel("timestamp")
plt.ylabel("target")
plt.legend()
plt.show()


✅ **Exercices (Chronos)**

1) Changez `PRED_LEN` (12, 24, 72) et observez.
2) Remplacez la série synthétique par un CSV : colonnes `timestamp`, `target`.
3) Essayez un modèle plus léger si besoin : `amazon/chronos-bolt-small`.


---
# Partie 2 — Moirai (Salesforce) via Uni2TS

Moirai s'utilise via la librairie **Uni2TS** et **GluonTS**.

Pipeline :
- convertir `pandas.DataFrame` en `PandasDataset`
- split train/test
- charger le checkpoint Moirai depuis Hugging Face
- créer un `predictor` et générer des forecasts


In [ ]:
# Installation
# (Si pip pose problème, essayez la version GitHub)
!pip -q install -U uni2ts gluonts huggingface_hub matplotlib
# !pip -q install -U "git+https://github.com/SalesforceAIResearch/uni2ts.git"


In [ ]:
import torch
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast

SIZE = "small"     # {'small','base','large'} selon vos ressources
PDT  = 48          # prediction length
CTX  = 200         # context length
PSZ  = "auto"      # {"auto", 8, 16, 32, 64, 128}
BSZ  = 32          # batch size
TEST = 96          # longueur du test

# Wide DataFrame -> GluonTS dataset
wide = df.set_index("timestamp")[["target"]]
ds = PandasDataset(dict(wide))

# Split train/test
train, test_template = split(ds, offset=-TEST)

# Rolling evaluation windows
test_data = test_template.generate_instances(
    prediction_length=PDT,
    windows=max(1, TEST // PDT),
    distance=PDT
)

# Télécharger le checkpoint
ckpt_path = hf_hub_download(
    repo_id=f"Salesforce/moirai-R-{SIZE}",
    filename="model.ckpt"
)

model = MoiraiForecast.load_from_checkpoint(
    checkpoint_path=ckpt_path,
    prediction_length=PDT,
    context_length=CTX,
    patch_size=PSZ,
    num_samples=200,
    target_dim=1,
    feat_dynamic_real_dim=ds.num_feat_dynamic_real,
    past_feat_dynamic_real_dim=ds.num_past_feat_dynamic_real,
    map_location="cuda:0" if torch.cuda.is_available() else "cpu",
)

predictor = model.create_predictor(batch_size=BSZ)
forecasts = list(predictor.predict(test_data.input))

# Afficher 1 exemple
inp = next(iter(test_data.input))
label = next(iter(test_data.label))
forecast = forecasts[0]

plot_single(inp, label, forecast, context_length=CTX, name="Moirai", show_label=True)


✅ **Exercices (Moirai)**

1) Essayez `SIZE='base'` si vous avez un GPU (sinon gardez `small`).
2) Changez `CTX` et `PDT`.
3) Comparez visuellement Chronos vs Moirai sur la même série (même horizon).


---
# Partie 3 — Time-MoE (Mixture-of-Experts)

Time-MoE est disponible via Hugging Face (`Maple728/TimeMoE-50M`, `TimeMoE-200M`, …) avec `trust_remote_code=True`.

Le modèle opère sur des séquences numériques **normalisées**. Ici on fait une normalisation simple (z-score) sur le contexte.


In [ ]:
# Installation
# Note: certains exemples Time-MoE recommandent transformers==4.40.1
!pip -q install -U "transformers==4.40.1" accelerate


In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_ID = "Maple728/TimeMoE-50M"  # petit modèle, plus léger

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map=DEVICE,
    trust_remote_code=True,
)

context_length = 200
prediction_length = 48

series = df["target"].values.astype(np.float32)
context = series[-context_length:]

# Normalisation simple
mu = context.mean()
sigma = context.std() + 1e-6
context_norm = (context - mu) / sigma

# Time-MoE attend typiquement un tensor [batch, context_len]
x = torch.tensor(context_norm[None, :], dtype=torch.float32)
x = x.to(model.device)

with torch.no_grad():
    out = model.generate(x, max_new_tokens=prediction_length)

pred_norm = out[:, -prediction_length:].detach().cpu().numpy().squeeze()
pred = pred_norm * sigma + mu

future_idx = pd.date_range(df["timestamp"].iloc[-1] + (df["timestamp"].iloc[1]-df["timestamp"].iloc[0]),
                           periods=prediction_length, freq="H")

plt.figure(figsize=(12, 3))
plt.plot(df["timestamp"].tail(200), df["target"].tail(200), label="historique")
plt.plot(future_idx, pred, label="prévision Time-MoE")
plt.title("Time-MoE — prévision (démo)")
plt.xlabel("timestamp")
plt.ylabel("target")
plt.legend()
plt.show()


✅ **Exercices (Time-MoE)**

1) Essayez `MODEL_ID="Maple728/TimeMoE-200M"` si vous avez un GPU.
2) Testez d'autres normalisations : min-max, robust (median/IQR).
3) Comparez les courbes Chronos / Moirai / Time-MoE sur la même figure.


---
# Bonus — Comparaison rapide (overlay)

On superpose Chronos (médiane) et Time-MoE si les variables existent.


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

plt.figure(figsize=(12, 3))
plt.plot(df["timestamp"].tail(200), df["target"].tail(200), label="historique")

# Chronos médiane
try:
    pred_chronos = pred_df.set_index("timestamp")["predictions"]
    plt.plot(pred_chronos.index, pred_chronos.values, label="Chronos-2 (médiane)")
except Exception as e:
    print("Chronos non disponible:", e)

# Time-MoE
try:
    plt.plot(future_idx, pred, label="Time-MoE")
except Exception as e:
    print("Time-MoE non disponible:", e)

plt.title("Comparaison visuelle")
plt.xlabel("timestamp")
plt.ylabel("target")
plt.legend()
plt.show()


## Fin

Vous pouvez maintenant :

- remplacer `df` par vos données réelles
- tester d'autres horizons et contextes
- ajouter des métriques (MAE, RMSE, MAPE, …) sur un vrai split train/test
